<a href="https://colab.research.google.com/github/miguel-peralta/cars_ista322/blob/main/cars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cars Relational Databases
ISTA 322 Final Project, Spring 2024 <br>
Miguel Candido Aurora Peralta <br>
## Extract
### KBB Web Scraping
The cars are separated into new and used cars categories. The lists new and used cars are on separate pages. These lists will be combined into one for this project.
#### Used Cars



In [11]:
# Imports
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [6]:
def get_html_doc(url):
    # requests HTML document for URL
    response = requests.get(url)
    # returns JSON response
    return response.text

In [ ]:
def get_models_df(url)

In [7]:
used_url = 'https://www.kbb.com/car-make-model-list/used/view-all/model/'
used_html = get_html_doc(used_url)
used_soup = BeautifulSoup(used_html, 'html.parser')


In [31]:
# Get all links on the page
# All elements with links have the attribute target="_self"
links = []
for link in used_soup.find_all('a',
                          attrs={'target': '_self'}):
    # Add links to the list
    links.append(link.get('href'))

In [40]:
links[41].split('/')

['', 'mercedes-benz', 'a-class', '2021', '']

In [50]:
# Links to car models start at index 40, last 17 are footer links
used_car_links = links[40:-17]
# Create dataframe and lists to store info from split links
used_car_info = pd.DataFrame(columns=['make', 'model', 'year'])
urls = []
make = []
model = []
year = []

base_url = 'https://www.kbb.com'

# Split links using / as delimeter and add information to lists
for car in used_car_links:
  urls.append(base_url+car)
  link_split = car.split('/')
  make.append(link_split[1])
  model.append(link_split[2])
  year.append(link_split[3])

# Use lists to populate dataframe
used_car_info['url'] = urls
used_car_info['make'] = make
used_car_info['model'] = model
used_car_info['year'] = year

In [52]:
used_car_info.tail()

,make,model,year,url
11276,oldsmobile,98,1996,https://www.kbb.com/oldsmobile/98/1996/
11277,oldsmobile,98,1995,https://www.kbb.com/oldsmobile/98/1995/
11278,oldsmobile,98,1994,https://www.kbb.com/oldsmobile/98/1994/
11279,oldsmobile,98,1993,https://www.kbb.com/oldsmobile/98/1993/
11280,oldsmobile,98,1992,https://www.kbb.com/oldsmobile/98/1992/
